___

# Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## Python-ogr vectors


Thi notebook introduces how to work with vector data in Python using ogr/gdal binding. It covers: 

* 1. Reading vector data; 

* 2. Cycling over features; 

* 3. Creating a new data source and layer; 


### Setup

In [4]:
# Common imports for reading, visualizing
import os
import sys
import matplotlib.pyplot as plt 
%matplotlib inline 

# import ogr
from osgeo import ogr

# Set your own PATH!!! 
PATH = './data/'

if os.path.isdir(PATH): 
    print('ok')

ok


### Get data 

In [5]:
vector_fn = os.path.join(PATH, 'polygons.shp')

In [6]:
if os.path.isfile(vector_fn): 
    print('ok')
else: 
    print('Check your path to the vector file!')

ok


### Reading vector

In [7]:
# get the shapefile driver
driver = ogr.GetDriverByName('ESRI Shapefile')

In [8]:
driver

<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x1141197e0> >

In [9]:
# open the data source
datasource = driver.Open(vector_fn, 0)
if datasource is None:
  print('Could not open file')
  sys.exit(1)

In [8]:
datasource

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x11df09a50> >

In [9]:
# get the data layer
layer = datasource.GetLayer()

In [10]:
layer

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x11df63c60> >

In [11]:
# get metadata
pass

In [12]:
# loop through the features in the layer

feature = layer.GetNextFeature()
i = 0
while feature:
    i += 1
    print(i)
    # get the attributes
    # label_ = feature.GetFieldAsString('label')
    note_ = feature.GetFieldAsString('note')
    print(note_)
    
    # get the x,y coordinates
    # geom = feature.GetGeometryRef()
    # x = str(geom.GetX())
    # y = str(geom.GetY())
    
    # destroy the feature and get a new one
    feature.Destroy()
    feature = layer.GetNextFeature()


1
glacier
2
debries
3
rock


In [13]:
# close the data source and text file
datasource.Destroy()

### Creating vector (polygon centroids) 

In [34]:
os.getcwd()

'/Users/lukas/Work/prfuk/vyuka/machine_learning_geosciences/src/geopython'

In [25]:
# os.chdir(...)

In [20]:
driver = ogr.GetDriverByName('ESRI Shapefile')

In [21]:
# open the input data source and get the layer
inDS = driver.Open(vector_fn, 0)
if inDS is None:
  print('Could not open file')
  sys.exit(1)
inLayer = inDS.GetLayer()

In [22]:
inLayer

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x1273e3e40> >

In [23]:
# create a new data source and layer

fn = os.path.join(PATH, 'polygon_centroid.shp') 
if os.path.exists(fn):
  driver.DeleteDataSource(fn)
outDS = driver.CreateDataSource(fn)
if outDS is None:
  print('Could not create file')
  sys.exit(1)
outLayer = outDS.CreateLayer(fn, geom_type=ogr.wkbPoint)

In [24]:
# get the FieldDefn's for the existing fields in the input shapefile
feature = inLayer.GetFeature(0)
labelFieldDefn = feature.GetFieldDefnRef('label')
noteFieldDefn = feature.GetFieldDefnRef('note')

In [25]:
labelFieldDefn

<osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x114173570> >

In [26]:
# create new fields in the output shapefile
outLayer.CreateField(labelFieldDefn)
outLayer.CreateField(noteFieldDefn)

0

In [27]:
# get the FeatureDefn for the output layer
featureDefn = outLayer.GetLayerDefn()

In [28]:
# inFeature = inLayer.GetNextFeature()

In [29]:
# geom = inFeature.GetGeometryRef()

In [30]:
# centroid = geom.Centroid()

In [31]:
# print(centroid.GetX(), centroid.GetY())

In [32]:
# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
    # get the cover attribute for the input feature
    label = inFeature.GetField('label')
    
    # create a new feature
    outFeature = ogr.Feature(featureDefn)

    # set the geometry
    geom = inFeature.GetGeometryRef()
    centroid = geom.Centroid()
    centroid.GetX()
    centroid.GetY()
    
    outFeature.SetGeometry(centroid)

    # set the attributes
    note = inFeature.GetField('note')
    outFeature.SetField('label', label)
    outFeature.SetField('note', note)

    # add the feature to the output layer
    outLayer.CreateFeature(outFeature)

    # destroy the output feature
    outFeature.Destroy()
    
    # destroy the input feature and get a new one
    inFeature.Destroy()
    inFeature = inLayer.GetNextFeature()

In [33]:
# close the data sources
inDS.Destroy()
outDS.Destroy()